# Figure 3

## Imports

In [ ]:
import dolfin
import matplotlib.pyplot as plt
import numpy

import dolfin_mech as dmech

## Parameters

### Mesh

In [ ]:
cube_params = {"path_and_mesh_name":"Meshes/generic_lung.xdmf"}

### Material

In [ ]:
alpha_lst = [0.016, 0.16, 1.6] # MG: Units!

params = {
    "alpha": 0.16, # MG: Units!
    "gamma":0.5, # MG: Units!
    "c1":0.6, # MG: Units!
    "c2":0., # MG: Units!
    "kappa":1e2, # MG: Units!
    "eta":1e-5, # MG: Units!
    "rho_solid":1.06e-6} # MG: Units!

mat_params = {"scaling":"linear", "parameters":params}

### Loading

In [ ]:
pi_lst = -numpy.linspace(0.5, 2., 21) # MG: Units!

pe = -0.5 # MG: Units!

g = 9.81e3 # MG: Units!

## Computing pressure-volume responses

In [ ]:
results = {}

gravity_lst = [0, 1]
for gravity_ in gravity_lst:
    for alpha_ in alpha_lst:

        mat_params["parameters"]["alpha"] = alpha_
        load_params_inverse = {
            "type":"p_boundary_condition0", "f":-gravity_*g, "P0":float(pe)}
        load_params_direct_exhalation = {
            "type":"p_boundary_condition", "f":-gravity_*g, "P0":float(pe)}
        load_params_direct_inhalation= {
            "type":"p_boundary_condition", "f":-gravity_*g, "P0":float(pi_)}

        J_lst = []
        for pi_ in pi_lst:

            ### computing the unloaded configuration
            U_exhal_to_unloaded, phis_unloaded, dV_exhal, dV_unloaded = dmech.run_RivlinCube_PoroHyperelasticity(
                inverse=1,
                cube_params=cube_params,
                mat_params=mat_params,
                load_params=load_params_inverse,
                inertia=1,
                step_params={"dt_ini":0.125, "dt_min":1e-4},
                res_basename="Fig3-inverse",
                get_results=1,
                verbose=1)
            
            V_exhal = dolfin.assemble(dolfin.Constant(1.) * dV_exhal)

            ### computing the end-inhalation configuration
            U_unloaded_to_inhal, phis_inhal, dV_unloaded, dV_inhal = dmech.run_RivlinCube_PoroHyperelasticity(
                inverse=0,
                cube_params=cube_params,
                move_params={"move":True, "U":U_exhal_to_unloaded},
                porosity_params={"type": "function_xml_from_array", "val":phis_unloaded},
                mat_params=mat_params,
                load_params=load_params_direct_inhalation,
                inertia=1,
                step_params={"dt_min":1e-4}, 
                res_basename="Fig3-direct",
                get_results=1,
                verbose=1)
            
            V_inhal = dolfin.assemble(dolfin.Constant(1.) * dV_inhal)
            J_def = V_inhal/V_exhal
            J_lst.append(J_def)

        results["J_alpha="+str(alpha_)+"g="+str(gravity_)] = J_lst

results["pi"] = pi_lst

## Generating plots

In [ ]:
fig, ax = plt.subplots()
plt.rc("xtick", labelsize=14)
plt.rc("ytick", labelsize=14)
plt.rc("legend", fontsize=12)
plt.xlabel("End-inhalation pleural presure", fontsize=10)
plt.ylabel("Jacobian from end-exhalation to end-inhalation", fontsize=10)
color_lst = ["blue", "green", "orange"]
for alpha_ in alpha_lst:
    for gravity_ in gravity_lst:
        if gravity_:
            linestyle = "-"
            g = "w/ gravity"
        else:
            linestyle = "--"
            g = "w/o gravity"
        plt.plot(results["pi"], results["J_alpha="+str(alpha_)+"g="+str(gravity_)], color=color_lst[0], linestyle=linestyle, label=r"$\alpha$"+"="+str(alpha_)+" kPa, "+str(g))
    color_lst=color_lst[1:]
plt.legend()
plt.grid()
plt.show()